## Подключаем необходимые модули

In [1]:
import requests
import json
from bs4 import BeautifulSoup

## Берем разметку сайта в формате HTML

In [2]:
web_page = '''http://www.banki.ru/banks/ratings/'''
r = requests.get(web_page)
print(r.ok)

True


## Преобразовываем в BS4 для удобства работы и извлечения данных

In [3]:
data = BeautifulSoup(r.text, "html.parser")

In [5]:
#data

## Собираем массив регионов для того, чтобы можно было спарсить отдельные таблицы для каждого региона.

In [6]:
data_region_all = data.findAll("script", attrs={"id":"region-popup-template"})
data_region_all = BeautifulSoup(data_region_all[0].text,"html.parser")
data_region_all = data_region_all.findAll('a', attrs={"class":"rating-parameter-list--item__link "})
#data_region_all

In [7]:
# Реализация массива href для регионов
data_region_href = {}
for i in data_region_all:
    data_region_href[i.text.strip()] = i['href']

In [8]:
#Очистка параметров
for key in data_region_href:
    data_region_href[key] = data_region_href[key].replace("/banks/ratings/?","")

In [8]:
#data_region_href

## Собираем массив показателей для реализации парсинга по отдельным показателям

In [9]:
data_indicator_all = data.findAll("script", attrs={"id":"indicator-popup-template"})
data_indicator_all = BeautifulSoup(data_indicator_all[0].text,"html.parser")
data_indicator_all = data_indicator_all.findAll('a', attrs={"class":"rating-parameter-list--item__link active"})
#data_indicator_all

In [10]:
# Реализация массива href для регионов
data_indicator_href = {}
for i in data_indicator_all:
    data_indicator_href[i.text.strip()] = i['href']

In [11]:
#Очистка параметров
for key in data_indicator_href:
    data_indicator_href[key] = data_indicator_href[key].replace("/banks/ratings/?","")

In [12]:
#data_indicator_href

## Собираем всевозможные даты для будущего парсинга

In [13]:
date = data.findAll("th", attrs={"class":"table-title font-normal"})
date_array = {}
for i in date[0].find_all('option'):
    date_array[i.text] = i["value"]

In [14]:
#date_array

## Все предврительные данные собрали, теперь можно и парсить начать

### введите следующие данные:

In [15]:
date_array.keys()

dict_keys(['Октябрь, 2018', 'Сентябрь, 2018', 'Август, 2018', 'Июль, 2018', 'Июнь, 2018', 'Май, 2018', 'Апрель, 2018', 'Март, 2018', 'Февраль, 2018', 'Январь, 2018', 'Декабрь, 2017', 'Ноябрь, 2017', 'Октябрь, 2017', 'Сентябрь, 2017', 'Август, 2017', 'Июль, 2017', 'Июнь, 2017', 'Май, 2017', 'Апрель, 2017', 'Март, 2017', 'Февраль, 2017', 'Январь, 2017', 'Декабрь, 2016', 'Ноябрь, 2016', 'Октябрь, 2016', 'Сентябрь, 2016', 'Август, 2016', 'Июль, 2016', 'Июнь, 2016', 'Май, 2016', 'Апрель, 2016', 'Март, 2016', 'Февраль, 2016', 'Январь, 2016', 'Декабрь, 2015', 'Ноябрь, 2015', 'Октябрь, 2015', 'Сентябрь, 2015', 'Август, 2015', 'Июль, 2015', 'Июнь, 2015', 'Май, 2015', 'Апрель, 2015', 'Март, 2015', 'Февраль, 2015', 'Январь, 2015', 'Декабрь, 2014', 'Ноябрь, 2014', 'Октябрь, 2014', 'Сентябрь, 2014', 'Август, 2014', 'Июль, 2014', 'Июнь, 2014', 'Май, 2014', 'Апрель, 2014', 'Март, 2014', 'Февраль, 2014', 'Январь, 2014', 'Декабрь, 2013', 'Ноябрь, 2013', 'Октябрь, 2013', 'Сентябрь, 2013', 'Август, 2013'

In [16]:
#С какой даты по какую нужны данные:
date_from1 = "Март, 2008"
date_to1 = "Октябрь, 2018"

if date_from1 != "": date_from = date_array[date_from1]
if date_to1 != "": date_to = date_array[date_to1]

---

In [17]:
data_region_href.keys()

dict_keys(['Москва и обл.', 'Санкт-Петербург и обл.', 'Адыгея', 'Алтайский край', 'Амурская обл.', 'Астраханская обл.', 'Башкортостан', 'Белгородская обл.', 'Владимирская обл.', 'Волгоградская обл.', 'Вологодская обл.', 'Дагестан', 'Ивановская обл.', 'Иркутская обл.', 'Кабардино-Балкария', 'Калининградская обл.', 'Калужская обл.', 'Камчатская обл.', 'Карачаево-Черкесия', 'Карелия', 'Кемеровская обл.', 'Кировская обл.', 'Коми', 'Костромская обл.', 'Краснодарский край', 'Красноярский край', 'Курганская обл.', 'Курская обл.', 'Липецкая обл.', 'Марий Эл', 'Мордовия', 'Мурманская обл.', 'Нижегородская обл.', 'Новгородская обл.', 'Новосибирская обл.', 'Омская обл.', 'Оренбургская обл.', 'Пензенская обл.', 'Пермская обл.', 'Приморский край', 'Псковская обл.', 'Республика Алтай', 'Россия', 'Ростовская обл.', 'Рязанская обл.', 'Самарская обл.', 'Саратовская обл.', 'Саха (Якутия)', 'Сахалинская обл.', 'Свердловская обл.', 'Севастополь', 'Северная Осетия — Алания', 'Симферополь', 'Ставропольский 

In [19]:
#Выберите регион:
region = ""

if region != "": region = data_region_href[region]

___

In [20]:
data_indicator_href.keys()

dict_keys(['Чистая прибыль', 'Капитал (по форме 123)', 'Капитал (по форме 134)', 'Кредитный портфель', 'Просроченная задолженность в кредитном портфеле', 'Вклады физических лиц', 'Вложения в ценные бумаги', 'Рентабельность активов-нетто', 'Рентабельность капитала', 'Уровень просроченной задолженности по кредитному портфелю', 'Уровень резервирования по кредитному портфелю', 'Уровень обеспечения кредитного портфеля залогом имущества', 'Валютный оборот к активам-нетто', 'Оборот по валютным операциям в тыс. рублей', 'Н1', 'Н2', 'Н3', 'Активы нетто', 'Высоколиквидные активы', 'Денежные средства в кассе', 'Денежные средства в кассе оборот', 'НОСТРО-счета', 'Кредиты физическим лицам', 'Сроком до 180 дней', 'Сроком от 181 дня до 1 года', 'Сроком от 1 года до 3 лет', 'Сроком более 3 лет', 'Овердрафты и прочие предоставленные средства', 'Просроченная задолженность', 'Выданные МБК', 'Размещенные МБК в ЦБ РФ', 'Размещенные МБК в ЦБ РФ оборот', 'Выданные МБК оборот всего', 'Кредиты предприятиям и о

In [21]:
#Выберите показатель:
indicator1 = "Капитал (по форме 123)"

if indicator1 != "": indicator = data_indicator_href[indicator1]

### да начнется магия

#### соберем страничку для парсинга

In [22]:
def web_page_for_parser(w_p, reg="",ind="", d_f="", d_t=""):
    param_ar = []
    if reg!="": param_ar.append(reg)
    if ind!="": param_ar.append(ind)
    if d_f!="": param_ar.append(("date2="+d_f))
    if d_t!="": param_ar.append(("date1="+d_t))
    ind_and = len(param_ar) - 1
    
    if len(param_ar)==0:
        return w_p
    
    w_p = w_p + '?'
    for par in param_ar:
        w_p += par
        if ind_and>0:
            w_p += "&"
            ind_and -=1
    
    return w_p


In [23]:
w_p_pars = web_page_for_parser(web_page,region,indicator,date_from,date_to)
w_p_pars

'http://www.banki.ru/banks/ratings/?PROPERTY_ID=850&date2=2008-03-01&date1=2018-10-01'

In [24]:
pars = requests.get(w_p_pars)
print(pars.ok)

True


#### Ну вот и самое интересное - парсинг

функция (костыль) для очисти первого столбца (места в рейтинге) от ненужных символов

In [25]:
def subnum(lstr):
    nstr = ''
    if lstr.isdigit() == False:
        for i in range(len(lstr)):
            if lstr[i].isdigit() == True:
                nstr += lstr[i]
            else:
                return nstr
    
    return lstr

а вот и сам парсер

In [26]:
i=1  # счетчик страниц
table = []  # итоговая таблица
while(True):  # цикл обработки информации
    FLAG = True  # наш флаг по которому проверяется смысл дальнейшей работы
    w_p_pars = w_p_pars + "&PAGEN_1={}".format(i)  # добавляем параметр страницы для сайта
    pars = requests.get(w_p_pars)  # получение html разметки в виде текста
    data_pars = BeautifulSoup(pars.text, "html.parser")  # преобразование в соуп для удобства обработки
    
    for dat in data_pars.findAll("tr", attrs={"data-test":"rating-table-item"}):  #цикл пробега по определенным элементам в разметке
        line = []
        line.append(subnum(dat.findAll('td')[0].text.strip()))  # Место
        line.append(dat.findAll('td')[1].find('a').text.strip())  # Имя банка
        line.append(dat.findAll('td')[1].findAll('div')[1].text.strip())  # Лицензия
        line.append(dat.findAll('td')[2].text.strip())  # Данные за Data_to
        line.append(dat.findAll('td')[3].text.strip())  # Данные за Data_from
        # проверка (потому как страницы по другому не проверить, или я не нашел)
        for j in table: # пробег по строкам нашей таблицы
            if line == j:  #проверка спарсеной строки на повтор
                FLAG = False
                break
        
        # выход из внутреннего цикла по флагу
        if FLAG == False:
            break
            
        table.append(line)  #добавление строки в таблицу
    
    w_p_pars = w_p_pars.replace("&PAGEN_1={}".format(i),'')  # убираем параметр страницы для сайта 
    
    # выход из внешнего цикла по флагу 
    if FLAG == False: 
        break
        
    i+=1  # приращение счетчика

In [37]:
#table

#### Теперь преобразуем данные в Табличный вид

In [29]:
import pandas as pd
import numpy as np

In [31]:
column_3_4 = data_pars.findAll("th", attrs={"class":"table-title is-center"})[0].text.split(",")[1].strip()
column_3_4

'тыс. рублей'

In [33]:
column_3 = indicator1 + " за {}, ".format(date_to1) + column_3_4
column_4 = indicator1 + " за {}, ".format(date_from1) + column_3_4
columns = ["Место в рейтинге", "Название Банка", "Лицензия (Регион)", column_3, column_4]

In [34]:
table_pd = pd.DataFrame(table, columns=columns)

In [35]:
table_pd.head()

,Место в рейтинге,Название Банка,Лицензия (Регион),"Капитал (по форме 123) за Октябрь, 2018, тыс. рублей","Капитал (по форме 123) за Март, 2008, тыс. рублей"
0,1,Сбербанк России,"лицензия № 1481, Москва и обл.",4 055 115 544,н/д
1,2,ВТБ,"лицензия № 1000, Санкт-Петербург и обл.",1 449 163 257,н/д
2,3,Газпромбанк,"лицензия № 354, Москва и обл.",735 244 449,н/д
3,4,Россельхозбанк,"лицензия № 3349, Москва и обл.",427 835 382,н/д
4,5,Альфа-Банк,"лицензия № 1326, Москва и обл.",425 511 598,н/д
